In [243]:
import pandas as pd
import numpy as np
import re

pd.options.display.max_columns = 999

In [257]:
df1 = pd.read_csv('../data/raw/amazon-walmart/orig-set1.csv', encoding = 'latin1')
df2 = pd.read_csv('../data/raw/amazon-walmart/orig-set2.csv', encoding = 'latin1')
matches = pd.read_csv('../data/raw/amazon-walmart/orig-matches.csv', encoding = 'latin1')

df1 = df1.loc[:, ~df1.columns.str.contains('orig')]
df2 = df2.loc[:, ~df2.columns.str.contains('orig')]

In [258]:
def pick_price(x):
    if np.isnan(x[0]):
        x[0] = x[1]
    return x
df1['listprice'] = df1[['listprice','price']].apply(pick_price, axis = 1)['listprice']

def pick_weight(x):
    #print(x[0], x[1], end = ', ')
    if not isinstance(x[0], str):
        if not isinstance(x[1], str):
            x[0] = 'nan'
        else:
            x[0] = x[1]
    return x

def weight_to_pounds(x):
    try:
        if re.search('ounces', x):
            x = re.sub('ounces', '', x)
            x = float(x)
            x /= 16
        elif re.search('pounds', x):
            x = re.sub('pounds', '', x)
            x = re.sub('\s|,', '', x)
            x = float(x)
        else:
            x = np.nan
    except:
        print(x)

df1['itemweight'] = df1[['itemweight','shipweight']].apply(pick_weight, axis = 1)['itemweight']
df1['itemweight'] = df1['itemweight'].apply(weight_to_pounds)

In [259]:
new_columns = ['id1', 'url', 'asin', 'brand', 'pcategory1', 'title', 'listprice',
               'techdetails', 'proddescrshort', 'proddescrlong', 'imageurl',
               'modelno', 'itemweight', 'dimensions']
df1 = df1[new_columns]

In [260]:
def convert_dimensions(x):
    if not isinstance(x, str):
        return [np.nan] * 3
    x = x.strip('inches').split('x')
    if len(x) != 3:
        return [np.nan] * 3
    return x

dimensions_1 = df1['dimensions'].apply(convert_dimensions)
data = list(zip(dimensions_1.index, dimensions_1))
dim_columns = ['length', 'width', 'height']
df1[dim_columns] = pd.DataFrame.from_items(data).T.astype(float)
df1 = df1.drop('dimensions', axis = 'columns')

dimensions_2 = df2['dimensions'].apply(convert_dimensions)
data = list(zip(dimensions_2.index, dimensions_2))
dim_columns = ['length', 'width', 'height']
df2[dim_columns] = pd.DataFrame.from_items(data).T.astype(float)
df2 = df2.drop('dimensions', axis = 'columns')

In [262]:
assert(df1.columns.size == df2.columns.size)

df1.to_csv('../data/raw/amazon-walmart/set1.csv', index=False)
df2.to_csv('../data/raw/amazon-walmart/set2.csv', index=False)
matches[['id1', 'id2']].to_csv('../data/raw/amazon-walmart/matches.csv', index=False)